In [1]:
import os
from pathlib import Path
import numpy as np
import cv2
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

In [2]:
BUNDLE_PATH = "svm_ocr_arialv2.pkl"  
bundle = joblib.load(BUNDLE_PATH)
clf = bundle["model"] if isinstance(bundle, dict) else bundle 

In [5]:
# --------- Preprocess (binary white-on-black + resize + flatten) ---------
def preprocess_tile_28(img, out_size=28):
    # img: np.ndarray (H,W) uint8 or float; expects grayscale image
    if img.ndim == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Hard binarize
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    # Ensure white=foreground, black=background
    if img.mean() > 127:
        img = 255 - img
    # Resize to model input
    img = cv2.resize(img, (out_size, out_size), interpolation=cv2.INTER_NEAREST)
    # Flatten to 1D vector (normalize to [0,1] if your model was trained that way)
    vec = (img.astype(np.float32) / 255.0).ravel()
    return vec

# --------- Label extraction helpers ---------
def infer_label_from_path(p: Path):
    """
    Priority 1: filename like '3_00012.png' -> 3
    Priority 2: parent folder name like .../3/img.png -> 3
    """
    name = p.name
    try:
        return int(name.split("_")[0])
    except Exception:
        return int(p.parent.name)


# --------- Evaluate a provided list of image paths ---------
def evaluate_image_list(clf, image_paths, img_size=28, verbose=True):
    """
    clf: trained scikit-learn model (e.g., LinearSVC / SVC) that expects raw pixels (flattened).
    image_paths: iterable of strings/Path pointing to images.
    img_size: expected input size (default 28).
    Returns: dict with 'accuracy', 'report', 'confusion_matrix', 'y_true', 'y_pred'
    """
    X, y_true, bad_paths = [], [], []

    for p in image_paths:
        p = Path(p)
        if not p.exists():
            bad_paths.append(str(p))
            continue
        im = cv2.imread(str(p), cv2.IMREAD_GRAYSCALE)
        if im is None:
            bad_paths.append(str(p))
            continue
        X.append(preprocess_tile_28(im, out_size=img_size))#preprocess_tile_hog(im, out_size=img_size))#
        try:
            y_true.append(infer_label_from_path(p))
        except Exception:
            bad_paths.append(str(p))
            X.pop()  # drop the sample if label failed
            continue

    if len(X) == 0:
        raise RuntimeError("No valid images to evaluate. Check paths/labels.")

    X = np.asarray(X, dtype=np.float32)
    y_true = np.asarray(y_true, dtype=np.int64)

    # Predict
    y_pred = clf.predict(X)

    # Metrics
    acc = accuracy_score(y_true, y_pred)
    report = classification_report(y_true, y_pred, digits=4)
    cm = confusion_matrix(y_true, y_pred)

    if verbose:
        print(f"[INFO] Evaluated {len(y_true)} images.")
        if bad_paths:
            print(f"[WARN] Skipped {len(bad_paths)} images (read/label errors).")
        print(f"[RESULT] Accuracy: {acc:.4f}")
        print("[RESULT] Classification report:\n", report)
        print("[RESULT] Confusion matrix:\n", cm)

    return {
        "accuracy": acc,
        "report": report,
        "confusion_matrix": cm,
        "y_true": y_true,
        "y_pred": y_pred,
        "skipped": bad_paths,
    }

# --------- Convenience: gather images recursively from a root folder ---------
def collect_images_from_root(root_dir, exts=(".png", ".jpg", ".jpeg"), limit_per_class=None):
    """
    Recursively collects image paths. If limit_per_class is set, limits samples per numeric subfolder.
    Returns a list of Paths.
    """
    root = Path(root_dir)
    per_class_count = {}
    imgs = []

    for p in root.rglob("*"):
        if not p.is_file() or p.suffix.lower() not in exts:
            continue
        # class from file or parent folder
        try:
            label = int(p.name.split("_")[0])
        except Exception:
            try:
                label = int(p.parent.name)
            except Exception:
                continue  # skip non-numeric classes

        if limit_per_class is not None:
            c = per_class_count.get(label, 0)
            if c >= limit_per_class:
                continue
            per_class_count[label] = c + 1

        imgs.append(p)

    return imgs

In [7]:
paths = collect_images_from_root(r"..\..\Data\dataset\text_arial_test2",
                                 exts=(".png", ".jpg"),
                                 limit_per_class=1)  # o 500, etc.

metrics = evaluate_image_list(clf, paths, img_size=28, verbose=True)
print(metrics)

[INFO] Evaluated 10 images.
[RESULT] Accuracy: 1.0000
[RESULT] Classification report:
               precision    recall  f1-score   support

           0     1.0000    1.0000    1.0000         1
           1     1.0000    1.0000    1.0000         1
           2     1.0000    1.0000    1.0000         1
           3     1.0000    1.0000    1.0000         1
           4     1.0000    1.0000    1.0000         1
           5     1.0000    1.0000    1.0000         1
           6     1.0000    1.0000    1.0000         1
           7     1.0000    1.0000    1.0000         1
           8     1.0000    1.0000    1.0000         1
           9     1.0000    1.0000    1.0000         1

    accuracy                         1.0000        10
   macro avg     1.0000    1.0000    1.0000        10
weighted avg     1.0000    1.0000    1.0000        10

[RESULT] Confusion matrix:
 [[1 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0